# LIBRERIAS

In [3]:
import pandas as pd
from google.cloud import bigquery
import pywhatkit
import time
import pyautogui
import time
import threading
from datetime import datetime, timedelta

# BIG QUERY

In [ ]:
# Conexión Google Cloud SDK
# https://cloud.google.com/appengine/docs/standard/setting-up-environment?tab=python&hl=es-419

# Ejecutar lo siguiente en Powershell
## (New-Object Net.WebClient).DownloadFile("https://dl.google.com/dl/cloudsdk/channels/rapid/GoogleCloudSDKInstaller.exe", "$env:Temp\GoogleCloudSDKInstaller.exe") & $env:Temp\GoogleCloudSDKInstaller.exe
## gcloud auth application-default login

In [4]:
# Especifica el ID del proyecto de Google Cloud
project_id = 'bi-fcom-drmb-local-pe-sbx'

# Crea una instancia del cliente de BigQuery con el ID del proyecto especificado
client = bigquery.Client(project=project_id)

# ID del conjunto de datos que contiene la tabla
dataset_id = 'Snorlax_SellerJourney'

# Nombre de la tabla a la que deseas acceder
table_id = 'sellers_registered'

# Consulta SQL para obtener datos de la tabla
query = f"""
        SELECT DATE(created_at) AS fecha_creacion,CONCAT('+51', Telefono) AS Telefono_Concat, Activos,src_id, name, correo_seller,origen,account_status,
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE account_status ='Activo'
        AND origen ='nacional'
"""

# Ejecutar la consulta y obtener los resultados
query_job = client.query(query)
    
# Convertir los resultados a un DataFrame
df = query_job.to_dataframe()

# Mostrar el DataFrame
print(df)

c:\Users\jalopezz\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


      fecha_creacion Telefono_Concat  Activos   src_id                  name  \
0         2022-09-26    +51940163325       72  SC382A6             DATO S.A.   
1         2024-01-26    +51933350638     1291  SCCBAAA             Modelarq.   
2         2023-04-25    +51994134666      156  SCB4AC8  TIENDA OFICIAL BOSCH   
3         2023-01-18    +51950746561       30  SCB2B9D   CROSLAND AUTOMOTRIZ   
4         2023-09-20    +51914909019      263  SCC9A55        AION HOME PERU   
...              ...             ...      ...      ...                   ...   
10018     2022-09-21    +51969959450       29  SC81941               Evaflex   
10019     2023-11-30    +51960241218       29  SC2B98C            SuperVenta   
10020     2022-11-15    +51987806150       29  SC957F6   Lomo Plateado Store   
10021     2023-11-06    +51940624134       29  SC00526    LA TABERNA DE BACO   
10022     2023-09-14    +51981071976       29  SC7DD9D                   KYM   

                                     co

# VARIABLE DE DÍAS

In [ ]:
# Obtener la fecha de ayer, antes de ayer y antes pasado ayer
ayer = datetime.today().date() - timedelta(days=1)
ayer_str = ayer.strftime('%Y-%m-%d')

antes_de_ayer = ayer - timedelta(days=1)
antes_de_ayer_str = antes_de_ayer.strftime('%Y-%m-%d')

antes_antes_de_ayer = antes_de_ayer - timedelta(days=1)
antes_antes_de_ayer_str = antes_antes_de_ayer.strftime('%Y-%m-%d')

# CONDICIONALES 

In [ ]:
# Crear DataFrame y especificar que 'Activos' y 'Telefono' son de tipo str
info_data = pd.DataFrame(df, columns=["fecha_creacion","Activos", "src_id", "Telefono_Concat", "name", "correo_seller","origen","account_status"], dtype=str)

# Filtrar el DataFrame para obtener solo las filas donde la fecha coincide con hoy
#info_data_bienvenida = info_data[info_data['fecha_creacion'].str.slice(0, 10) == ayer_str]

info_data_bienvenida = info_data[info_data['fecha_creacion'].between('2024-03-12', '2024-03-15')]

# Filtrar el DataFrame para obtener solo las filas donde 'Activos' es igual a '0'
info_data_refuerzo = info_data[(info_data['Activos'] == '0') & 
                               (info_data['fecha_creacion'].str.slice(0, 10) == antes_antes_de_ayer_str)]

# Visualizar los DataFrames resultantes
print("Datos de bienvenida:")
print(info_data_bienvenida)

print("\nDatos de refuerzo:")
print(info_data_refuerzo)

# ENVÍO DE WHATS APP

# 1. BIENVENIDA

In [ ]:
mensaje_whatsapp_1 = """
Hola {Seller}, te damos la bienvenida a falabella.com 👋  
 
Impulsa tus ventas online este Verano 2024 ¿Cómo? Publicando tu catálogo y colocando tus mejores ofertas.  
 
👉 Manual para crear productos: https://bit.ly/productosFSC  
 
👉 Costos y comisiones: https://bit.ly/CostosFSC  
 
Soy Vania, si tienes alguna duda estoy para apoyarte.  
 
🟠 Ingreso a falabella seller center: https://sellercenter.falabella.com/  
 
_Nos encantaría conocer un poco más sobre tu negocio, confírmanos el llenado de esta encuesta por favor_
 
Encuesta: https://forms.office.com/r/Qyqad6cgrF"""



for seller in info_data_bienvenida['src_id'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller = info_data_bienvenida[info_data_bienvenida['src_id'] == seller]
    
    Numero = df_seller['Telefono_Concat'].iloc[0]
    
    whatsapp_message = mensaje_whatsapp_1.format(
                    seller=df_seller["name"].iloc[0],                    
            )
    #Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(Numero, whatsapp_message)
    time.sleep(10)

# 2. REFUERZO 

In [ ]:
mensaje_whatsapp_1 = """
¡Hola {seller}! 
 
*Sabias que el 60% de clientes prefieren Retiro en tienda como medio de entrega* 📦 
  
Publica tu catálogo y activaremos esta opción para tu cuenta 🤩 
  
👉 Tutorial para crear productos: https://bit.ly/Tutoriales-FalabellaSellers 
  
Soy Diego, si tienes alguna consulta no dudes en escribirme.  
  
🟠 Ingreso a falabella seller center: https://sellercenter.falabella.com/   
"""



for seller in info_data_refuerzo['src_id'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_1 = info_data_refuerzo[info_data_refuerzo['src_id'] == seller]
    
    Numero = df_seller_1['Telefono_Concat'].iloc[0]
    
    whatsapp_message = mensaje_whatsapp_1.format(
                    seller=df_seller_1["name"].iloc[0],                    
            )
    #Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(Numero, whatsapp_message)
    time.sleep(10)   